In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import copy
import gdown
import threading
import importlib
import sys
from tqdm.notebook import tqdm
import ctypes

# Competição de AI's para o jogo de Othello

A atividade final do curso de aprendizado por reforço é uma compedição de AI's para o jogo de Othello. Reveja as regras do jogo
[neste notebook](https://colab.research.google.com/drive/14vsUbeO1d8K-PSxCagtRH7DYm68S-AB0?usp=sharing).

Novamente, a classe ```Othello``` implementa as regras do jogo.

In [ ]:
class Othello():

  direcoes_captura =\
   [(-1, -1), (-1, 0), (-1, +1), (0, -1), (0, +1), (+1, -1), (+1, 0), (+1, +1)]

  def __init__(self, outro = None, estado = None):
    if estado:
      self._carrega_estado(estado)
    elif outro:  # Construtor de cópia
      self._cols = outro._cols
      self._lins = outro._lins
      self._jogador_atual = outro._jogador_atual
      self._tabuleiro = copy.deepcopy(outro._tabuleiro)
      self._terminou = outro._terminou
      self._placar = list(outro._placar)
      # Normalmente essa informação vai ser descartada, então não é copiada
      self._capturas = None
      self._jogadas_legais = None
    else: # Novo jogo
      self._cols = 8
      self._lins = 8
      self._reset()

  def _reset(self):
    self._jogador_atual = 1
    self._tabuleiro = [\
      [ 0, 0, 0, 0, 0, 0, 0, 0],\
      [ 0, 0, 0, 0, 0, 0, 0, 0],\
      [ 0, 0, 0, 0, 0, 0, 0, 0],\
      [ 0, 0, 0,-1, 1, 0, 0, 0],\
      [ 0, 0, 0, 1,-1, 0, 0, 0],\
      [ 0, 0, 0, 0, 0, 0, 0, 0],\
      [ 0, 0, 0, 0, 0, 0, 0, 0],\
      [ 0, 0, 0, 0, 0, 0, 0, 0]\
    ]
    self._terminou = False
    self._placar = [2,2]
    self._capturas = None
    self._jogadas_legais = None

  def dim(self):
    return (self._cols, self._lins)

  def jogador_atual(self):
    self._checa_estado_atualizado()
    return self._jogador_atual

  # Retorna o conteúdo do tabuleiro em uma posicao
  def posicao(self, posicao):
    return self._tabuleiro[posicao[0]][posicao[1]]

  # Retorna uma cópia do tabuleiro
  def tabuleiro(self):
    return np.array(self._tabuleiro, dtype=np.int8)

  def placar(self, jogador):
    return self._placar[(1+jogador)//2]

  # O estado só é completamente computado caso seja necessário
  # pois algumas jogadas podem ser descartadas
  def _checa_estado_atualizado(self):
    if self._jogadas_legais is None:
      self._atualiza_capturas_e_jogadas_legais()
      # Verifica se jogador atual tem jogadas disponíveis
      if len(self._jogadas_legais)==0:
        # Troca de jogador
        self._jogador_atual = -self._jogador_atual
        self._atualiza_capturas_e_jogadas_legais()
        # Jogo terminou?
        if len(self._jogadas_legais)==0:
          self._terminou = True

  # Verdadeiro se o jogo terminou, falso caso contrário
  def terminou(self):
    self._checa_estado_atualizado()
    return self._terminou

  # Verifica se uma determinada posicao é valida
  def posicao_valida(self, posicao):
    return posicao[0]>=0 and posicao[0]<self._lins and posicao[1]>=0 and posicao[1]<self._cols

  # Retorna a lista de capturas em uma determinada direção
  def _lista_de_capturas(self, posicao, direcao, jogador):
    posicoes = []
    lin = posicao[0] + direcao[0]
    col = posicao[1] + direcao[1]
    while self.posicao_valida((lin, col)) and self.posicao((lin, col)) == -jogador:
      posicoes.append((lin, col))
      lin += direcao[0]
      col += direcao[1]
    return posicoes if self.posicao_valida((lin, col)) and self.posicao((lin, col)) == jogador else []

  # Atualiza a lista de jogadas legais
  def _atualiza_capturas_e_jogadas_legais(self):
    self._jogadas_legais = set()
    self._capturas = []
    for i in range(self._lins):
      self._capturas.append([])
      for j in range(self._cols):
        cap_possivel = False
        if self._tabuleiro[i][j]==0:
          self._capturas[-1].append([])
          for d in Othello.direcoes_captura:
            self._capturas[-1][-1].append(self._lista_de_capturas((i, j), d, self._jogador_atual)            )
            cap_possivel = cap_possivel or len(self._capturas[-1][-1][-1])>0
        else:
          self._capturas[-1].append([[]]*len(Othello.direcoes_captura))
        if cap_possivel:
          self._jogadas_legais.add((i, j))

  # Verifica se uma jogada é legal
  def jogada_legal(self, jogada):
    self._checa_estado_atualizado()
    return jogada in self._jogadas_legais

  # Retorna o conjunto de jogadas legais
  def jogadas_legais(self):
    self._checa_estado_atualizado()
    return list(self._jogadas_legais)

  # Processa a captura de peças após uma jogada com base em listas de capturas
  def _processa_captura_pecas(self, listas_de_capturas):
    for lista_capturas in listas_de_capturas:
      for pi, pj in lista_capturas:
          self._tabuleiro[pi][pj] = self._jogador_atual
          self._placar[(1+self._jogador_atual)//2] += 1
          self._placar[(1-self._jogador_atual)//2] -= 1

  # Aplica uma jogada. Precisa de uma lista de capturas a ser executada
  def _aplica_jogada(self, jogada, listas_de_capturas):
    self._tabuleiro[jogada[0]][jogada[1]] = self._jogador_atual
    self._placar[(1+self._jogador_atual)//2] += 1
    self._processa_captura_pecas(listas_de_capturas)
    self._jogador_atual = -self._jogador_atual
    self._atualiza_capturas_e_jogadas_legais()
    # Verifica se jogador atual tem jogadas disponíveis
    if len(self._jogadas_legais)==0:
        # Troca de jogador
        self._jogador_atual = -self._jogador_atual
        self._atualiza_capturas_e_jogadas_legais()
        # Jogo terminou?
        if len(self._jogadas_legais)==0:
          self._terminou = True

  # Aplica jogada
  # Retorna um *novo jogo*
  def joga(self, jogada):
    self._checa_estado_atualizado()
    if self._terminou:
      raise RuntimeError("Jogo encerrado")
    if not self.jogada_legal(jogada):
      raise RuntimeError("Jogada Ilegal")
    # Clona jogo atual
    novo_jogo = Othello(self)
    novo_jogo._aplica_jogada(jogada, self._capturas[jogada[0]][jogada[1]])
    # O jogo é retornado em um estado semi-computado
    return novo_jogo

Os métodos relevantes são:

> ```__init__(self, outro = None)```: Constrói um novo jogo. Caso o método receba outro jogo no parâmetro ```outro```, é criada uma cópia deste jogo.

> ```jogador_atual(self)```: Retorna o índice do jogador atual. Os índices possíveis são -1 e 1. O jogador inicial é o jogador de índice 1. Este valor deve ser desconsiderado caso o jogo tenha acabado. Vide método ```terminou()``` adiante.

> ```posicao(self, posicao)```: Retorna o conteúdo do tabuleiro em uma determinada posição. Posição é uma tupla com os índices da linha e coluna do tabuleiro (baseados em zero). O valor retornado é o índice do jogador que possui a ficha na posição indicada ou zero se a posição está vazia.

> ```dim(self)```: Retona as dimensões do tabuleiro. Esta implementação retorna sempre $8 \times 8$.

> ```tabuleiro(self)```: Retorna um *array* numpy com o conteúdo do tabuleiro.

> ```placar(self, jogador)```: Retorna o placar do jogador passado no parâmetro ```jogador``` (este valor deve ser -1 ou 1).

> ```terminou(self)```: Retorna ```True``` caso o jogo tenha acabado, ```False``` caso contrário. Nota: Caso o jogo tenha terminado, o valor retornado pelo método ```jogador_atual()``` deve ser desconsiderado.

> ```jogada_legal(self, jogada)```: Retorna ```True``` caso o jogador atual possa jogar uma ficha na posição indicada por ```jogada```, ```False``` caso contrário. Jogadas são tuplas com linha e coluna.

> ```jogadas_legais(self)```: Retorna uma sequência com a lista de todas as jogadas legais para o jogador atual.

> ```joga(self, jogada)```: Retorna o resultado da jogada descrita no parâmetro ```jogada```. Este parâmetro é uma tupla com a linha e coluna onde deve ser colocada a ficha. Note que este método *não* modifica o estado do objeto, mas retorna um *novo* jogo com o resultado da jogada. Assim, se por exemplo a variável ```jogo``` contém o estado *atual* do jogo e a variável ```jogada``` contém a próxima jogada a ser feita, a variável jogo deve ser atualizada da seguinte maneira:
```
jogo = jogo.joga(jogada)
```




Experimente as regras do jogo com uma partida iterativa:

In [ ]:
def partida_interativa(jogo):
  rows, cols = jogo.dim()

  buttons = [widgets.Button(value = i, description=' ',disabled=False,buttom_style='',layout={'width': '35px', 'height': '35px'}) for i in range(cols*rows)]
  estado = widgets.HBox([widgets.Label(""), widgets.HTML(" ")])
  placar = widgets.VBox([widgets.Label("Placar:"),widgets.HTML(" "), widgets.HTML(" ")])

  def atualiza_jogo(jog):
    for ii in range(cols*rows):
      i = ii//rows
      j = ii % rows
      e = jog.posicao((i, j))
      if e==0:
        buttons[ii].disabled = not jog.jogada_legal((i,j))
      elif e==-1:
        buttons[ii].disabled = True
        buttons[ii].style.button_color = 'white'
      else:
        buttons[ii].disabled = True
        buttons[ii].style.button_color = 'black'
      placar.children[1].value = "<div style='text-align: center;background-color:White; color:Black'>{}</div>".format(jog.placar(-1))
      placar.children[2].value = "<div style='text-align: center;background-color:Black; color:White'>{}</div>".format(jog.placar(1))
    if jog.terminou():
      estado.children[0].value="Jogo terminou"
      estado.children[1].value=""
    else:
      estado.children[0].value="Proximo Jogador: "
      estado.children[1].value="<div style='background-color:Black'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</div>" if jog.jogador_atual()>0 else "<div style='background-color:White'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</div>"

  def registra_jogada(jogada):
    nonlocal jogo
    jogo = jogo.joga(jogada)
    atualiza_jogo(jogo)

  for i, b in enumerate(buttons):
    b.on_click((lambda x: lambda b: registra_jogada(x))((i//rows, i%cols)))
  board = widgets.GridBox(buttons, layout=widgets.Layout(grid_template_columns="repeat("+str(cols)+", 40px)"))

  display(widgets.VBox([widgets.HBox([board, placar]),estado]))
  atualiza_jogo(jogo)

In [ ]:
partida_interativa(Othello())

## IAs + Robôs Manipuladores

Aqui foi implementado o código osquestrador do jogo entre IAs associadas a robôs manipuladores

In [ ]:
# 1) Instalar pelo PyPI (nome de pacote costuma usar hífen)
%pip install -U pmr-elirobots-driver

# 2) (opcional) conferir versão e que o import funciona
import pmr_elirobots_driver, sys
print("driver:", getattr(pmr_elirobots_driver, "__version__", "?"), "python:", sys.version)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append("/content/drive/MyDrive/TCC_Othello_Juiz")  # ajuste o caminho se for outro


In [ ]:
%%writefile /content/drive/MyDrive/TCC_Othello_Juiz/othello_robo_ponte.py

import os
import time
import math
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional
import requests
import traceback

# === Config da bridge (lidas do ambiente) ===
BRIDGE_BASE  = os.environ.get("BRIDGE_BASE", "https://viperish-pressuringly-janessa.ngrok-free.dev").rstrip("/")
BRIDGE_TOKEN = os.environ.get("BRIDGE_TOKEN", "")

# === PARÂMETROS GLOBAIS DE MOVIMENTO (Apenas para exportação ao Juiz) ===
SPEED_PTP = int(os.environ.get("MOVE_SPEED", "90"))
SPEED_LINEAR = int(os.environ.get("MOVE_SPEED_LINEAR", "250"))
STAGING_SPEED = 90
ACCEL_PTP = int(os.environ.get("MOVE_ACCEL", "80"))
ACCEL_LINEAR = int(os.environ.get("MOVE_ACCEL_LINEAR", "80"))

# --- PARÂMETROS DE STAGING (usado para trocar de lado) ---
STAGING_JOINTS = [70.0, -105.0, 110.0, -95.0, 90.0, -20.0]

def _headers():
    return {"Authorization": f"Bearer {BRIDGE_TOKEN}"} if BRIDGE_TOKEN else {}

def http_get(rid: int, path: str, **params):
    if not BRIDGE_BASE: raise RuntimeError("BRIDGE_BASE não definido.")
    url = f"{BRIDGE_BASE.rstrip('/')}{path}"
    all_params = {"rid": rid, **params}
    r = requests.get(url, headers=_headers(), params=all_params, timeout=120)
    r.raise_for_status(); return r.json()

def http_post(rid: int, path: str, json=None, **params):
    if not BRIDGE_BASE: raise RuntimeError("BRIDGE_BASE não definido.")
    url = f"{BRIDGE_BASE.rstrip('/')}{path}"
    all_params = {"rid": rid, **params}
    r = requests.post(url, headers=_headers(), json=json, params=all_params, timeout=600)
    r.raise_for_status(); return r.json()

IJ = Tuple[int, int]

# === Classe Historiadora ===
class Historiadora:
    def __init__(self, imprimir: bool = True, verbosidade: int = 2,
                 mostrar_matriz: bool = True, estilo: str = "ascii", largura_col: int = 1):
        self.eventos = []
        self.imprimir = imprimir
        self._seq = 0
        self.verbosidade = verbosidade
        self.mostrar_matriz = mostrar_matriz
        self.estilo = estilo
        self.largura_col = largura_col
        self.robo = None #
        self._ultima_jogada = None

    def _agora(self):
        import datetime
        return datetime.datetime.now().isoformat(timespec="seconds")
    def _out(self, s: str):
        if self.imprimir: print(s)
    def _mapa(self):
        if self.estilo == "unicode": return {1: "⬤", -1: "◯", 0: "·"}
        return {1: "X", -1: "O", 0: "."}
    def _tabuleiro_str(self, jogo, destaque=None):
        board = jogo.tabuleiro()
        mapa = self._mapa()
        w = self.largura_col
        linhas = []

        for i, linha in enumerate(board.tolist()):
            cells = []
            for j, v in enumerate(linha):
                ch = mapa[int(v)]
                if destaque is not None and (i, j) == tuple(destaque):
                    ch = f"\033[31m{ch}\033[0m"
                cells.append(f"{ch:<{w}}")
            linhas.append(" ".join(cells))
        return "\n".join(linhas)

    def _placar_de(self, jogo):
        return jogo.placar(+1), jogo.placar(-1)
    def _sep(self, titulo: str):
        barra = "═" * max(10, len(titulo) + 2)
        self._out(f"\n╔{barra}╗"); self._out(f"║ {titulo} ║"); self._out(f"╚{barra}╝")
    def registrar(self, tipo: str, **campos):
        self._seq += 1
        ev = {"seq": self._seq, "ts": self._agora(), "tipo": tipo, **campos}
        self.eventos.append(ev)
    def inicio_lance(self, jogador: int, jogada, jogo=None):
        self.registrar("inicio_lance", jogador=jogador, jogada=jogada)
        titulo = f"LANCE — jogador {jogador} — jogada {tuple(jogada)}"
        self._sep(titulo)
        self._ultima_jogada = tuple(jogada) if jogada is not None else None
        if jogo is not None:
            pretas, brancas = self._placar_de(jogo)
            if self.mostrar_matriz:
                self._out("")
                self._out(self._tabuleiro_str(jogo, destaque=self._ultima_jogada))
            self._out(f"Placar antes: ⬤ {pretas}  |  ◯ {brancas}")
    def fim_lance(self, modo: str, total_flips: int, jogo_antes=None, jogo_depois=None):
        self.registrar("fim_lance", modo=modo, total_flips=total_flips)
        self._out(f"Modo: {modo} | Peças viradas: {total_flips}")
        if (jogo_antes is not None) and (jogo_depois is not None):
            p0, b0 = self._placar_de(jogo_antes); p1, b1 = self._placar_de(jogo_depois)
            self._out(f"Placar: ⬤ {p0}→{p1}  |  ◯ {b0}→{b1}")
            if self.mostrar_matriz:
                self._out("Matriz depois:")
                self._out(self._tabuleiro_str(jogo_depois, destaque=self._ultima_jogada))
    def _rid(self): return self.robo.rid if self.robo else '?'
    def transicao_quadrante(self, de_q: str, para_q: str):
        pass
    def colocacao(self, casa, quadrante):
        pass
    def flip(self, casa, quadrante):
        pass
    def movimento(self, juntas, nome=None):
        pass
    def movimento_pose(self, pose, nome=None):
      pass
    def garra(self, acao: str):
        pass
    def mensagem(self, texto: str, **campos):
        rid = campos.get("rid", self._rid())
        self.registrar("mensagem", texto=texto, **campos)
        if self.verbosidade >= 1: self._out(f"ℹ [R{rid}] {texto}")

# === Classe Controladora ===
class ControladorRobo:
    def __init__(
        self,
        rid: int = 1,
        speed_ptp: int = SPEED_PTP,
        speed_linear: int = SPEED_LINEAR,
        accel_ptp: int = ACCEL_PTP,
        accel_linear: int = ACCEL_LINEAR,
        historico: Optional[Historiadora] = None,
        simulado: bool = False,
    ):
        self.rid = int(rid)
        self.h = historico
        if self.h: self.h.robo = self
        self.simulado = simulado
        self.speed_ptp = speed_ptp
        self.speed_linear = speed_linear
        self.accel_ptp = accel_ptp
        self.accel_linear = accel_linear

    def conectar(self):
        if self.simulado:
            if self.h: self.h.mensagem(f"conectar(simulado rid={self.rid})", rid=self.rid)
            return
        try:
            st = http_get(self.rid, "/status")
            if not st.get("ok"): raise RuntimeError(f"Robô rid={self.rid} indisponível: {st}")
            if self.h: self.h.mensagem(f"conectar(rid={self.rid})", status=st, rid=self.rid)
        except Exception as e:
            raise RuntimeError(f"Falha ao conectar rid={self.rid}: {e}") from e

    def habilitar_servos(self):
        if self.h: self.h.robo = self
        self._garante()
        if self.simulado:
            if self.h: self.h.mensagem(f"Habilitando servos (simulado rid={self.rid})")
            return
        try:
            if self.h: self.h.mensagem(f"Enviando comando /habilitar (rid={self.rid})...")
            res = http_post(self.rid, "/habilitar")
            if not res.get("ok"):
                raise RuntimeError(f"/habilitar rid={self.rid} falhou: {res}")
            if self.h: self.h.mensagem(f"Servos habilitados (rid={self.rid}).")
            time.sleep(1.0)
        except Exception as e:
            raise RuntimeError(f"Falha ao habilitar servos rid={self.rid}: {e}") from e

    def desabilitar_servos(self):
        if self.h: self.h.robo = self
        if self.simulado:
            if self.h: self.h.mensagem(f"Desabilitando servos (simulado rid={self.rid})")
            return
        try:
            if self.h: self.h.mensagem(f"Enviando comando /servo (on=False, rid={self.rid})...")
            res = http_post(self.rid, "/servo", on=False)
            if not res.get("ok"):
                if self.h: self.h.mensagem(f"Aviso: /servo rid={self.rid} falhou: {res}")
            if self.h: self.h.mensagem(f"Servos desligados (rid={self.rid}).")
        except Exception as e:
            if self.h: self.h.mensagem(f"Falha ao desligar servos rid={self.rid}: {e}")

    def _garante(self):
        if self.simulado: return
        st = http_get(self.rid, "/status")
        if not st.get("ok"): raise RuntimeError(f"Robô rid={self.rid} indisponível: {st}")

    def _exec_http(self, method_func, endpoint, payload, nome_log, log_data, **params):
        """Função genérica para chamadas HTTP, mantida para ir_home."""
        if self.h: self.h.robo = self
        if self.simulado:
            if "juntas" in log_data: self.h.movimento(**log_data)
            return
        try:
            rid_param = params.pop("rid", self.rid)
            res = method_func(rid_param, endpoint, json=payload, **params)
            if not res.get("ok"):
                raise RuntimeError(f"{endpoint} rid={self.rid} falhou: {res.get('result', res.get('detail', 'Erro desconhecido'))}")
            if "juntas" in log_data: self.h.movimento(**log_data)
        except requests.exceptions.RequestException as e:
            raise ErroRobo(f"Falha HTTP no {endpoint} (rid={self.rid}): {e}") from e

    def ir_home(self):
        log_data = {"juntas": "HOME", "nome": "home"}
        self._exec_http(http_post, "/home", None, "home", log_data)


# === Classe Orquestradora ===
class OrquestradorOthelloRobo:
    def __init__(self, robo1: ControladorRobo, robo2: ControladorRobo, historico: Optional[Historiadora] = None, usar_camera: bool = True):
        self.robos = {1: robo1, -1: robo2}
        self.h = historico
        self.usar_camera = usar_camera
        self._ultimo_modo = "?"
        self._ultimo_total = 0
        self.pecas_usadas = {1: 0, -1: 0}

    def _get_robo(self, jogador_atual: int) -> Tuple[ControladorRobo, int, int, int, int]:
        robo = self.robos.get(jogador_atual)
        if not robo:
            raise ValueError(f"Nenhum robô configurado para o jogador {jogador_atual}")
        return robo, robo.speed_ptp, robo.speed_linear, robo.accel_ptp, robo.accel_linear

    # --- Funções de Jogada Humana ---
    def preparar_para_jogada_humana(self, jogo, jogador_humano: int = -1):
        if not self.usar_camera:
            if self.h: self.h.mensagem("[WARN] Orquestrador em MODO CEGO. Jogada humana não terá detecção real.")
            return

        rid = self.robos[jogador_humano].rid
        if self.h: self.h.mensagem(f"[R{rid}][VIS] Enviando Snapshot Lógico para detecção.")

        try:
            tabuleiro_lista = jogo.tabuleiro().tolist()
            http_post(rid, "/vis/preparar_jogada_humano", json=tabuleiro_lista)
        except Exception as e:
            if self.h: self.h.mensagem(f"[R{rid}][VIS] ERRO ao preparar jogada: {e}")

    def checar_jogada_humano(self, jogador_humano: int = -1) -> Optional[Tuple[int, int]]:
        if not self.usar_camera:
            return None

        rid = self.robos[jogador_humano].rid
        try:
            response = http_get(rid, "/vis/get_jogada_humano")
            if response.get("status") == "JOGADA_PRONTA":
                jogada = response.get("data", {}).get("jogada")
                if jogada and len(jogada) == 2:
                    jogada_tuple = (int(jogada[0]), int(jogada[1]))
                    if self.h: self.h.mensagem(f"[R{rid}][VIS] Jogada humana recebida do bridge: {jogada_tuple}")
                    return jogada_tuple
            return None
        except Exception as e:
            if self.h: self.h.mensagem(f"[R{rid}][VIS] ERRO ao checar jogada: {e}")
            return None

    # --- Lógica Principal ---
    def executar_lance(self, jogo, jogada: IJ, capturas_por_direcao: List[List[IJ]], jogador_atual: int):

        robo_ativo, speed_ptp, speed_lin, accel_ptp, accel_lin = self._get_robo(jogador_atual)
        rid = robo_ativo.rid
        if self.h: self.h.robo = robo_ativo

        # === 1. MONTA O "PLANO DE VOO" PARA O DASHBOARD ===
        peca_id = self.pecas_usadas[jogador_atual]

        steps = []
        steps.append(f"Iniciar Sequência (Robô {rid})")
        steps.append(f"Pegar Peça N°{peca_id + 1} (Estojo)")
        steps.append(f"Colocar em {tuple(jogada)}")

        # Adiciona os flips na lista
        total_flips = 0
        for direcao in capturas_por_direcao:
            for (fi, fj) in direcao:
                steps.append(f"Virar Peça em ({fi}, {fj})")
                total_flips += 1

        steps.append(f"Retornar ao Home")

        # Envia o status COM a lista de passos
        nome_robo = "Robô 1 (Pretas)" if jogador_atual == 1 else "Robô 2 (Brancas)"
        try:
            http_post(rid, "/game/set_status", json={
                "jogador": nome_robo,
                "acao": f"Executando Jogada ({total_flips} flips)...",
                "jogada": list(jogada),
                "passos": steps
            })
        except: pass

        # ====================================================================
        # TRAVA DE SEGURANÇA (ON/OFF)
        # ====================================================================

        if self.usar_camera:
            # --- MODO VISUAL SEGURO ---
            if self.h: self.h.mensagem(f"[R{rid}][VIS] Verificando segurança visual antes de mover...")
            import time
            tempo_limite = 60
            inicio_espera = time.time()

            while True:
                try:
                    # 1. Prepara o tabuleiro esperado
                    tabuleiro_esperado = jogo.tabuleiro().tolist()

                    # 2. Pergunta ao Bridge
                    resp_validacao = http_post(rid, "/vis/validar_estado", json=tabuleiro_esperado)

                    if resp_validacao.get("ok"):
                        # SUCESSO: Tabuleiro limpo e correto. Sai do loop e joga.
                        if self.h: self.h.mensagem(f"[R{rid}][VIS] Área segura. Autorizando movimento.")
                        break

                    # FALHA
                    detalhes_erro = resp_validacao.get("detail", {}).get("erros", ["Erro desconhecido"])
                    msg_aviso = f"BLOQUEIO VISUAL: {detalhes_erro}. Corrija o tabuleiro!"

                    print(f"--- [AGUARDANDO] {msg_aviso} ---")
                    if self.h: self.h.mensagem(f"[R{rid}][VIS-WAIT] {msg_aviso}")

                    if (time.time() - inicio_espera) > tempo_limite:
                        raise ErroRobo(f"Timeout de Segurança: O tabuleiro não foi corrigido em {tempo_limite}s.")

                    time.sleep(2.0)

                except Exception as e:
                    if "Timeout de Segurança" in str(e): raise e
                    print(f"Erro de conexão na validação: {e}")
                    time.sleep(2.0)
                    if (time.time() - inicio_espera) > tempo_limite:
                         raise ErroRobo(f"Timeout: Falha de comunicação com a câmera/bridge.")
        else:
            # --- MODO CEGO ---
            if self.h: self.h.mensagem(f"[R{rid}][CEGO] Validação de câmera DESLIGADA. Movendo cegamente...")
            time.sleep(0.5) # Pequeno delay para garantir que o outro robô liberou o lock se foi rápido

        # ====================================================================
        # FIM DA TRAVA DE SEGURANÇA
        # ====================================================================

        num_peca_atual = self.pecas_usadas[jogador_atual]

        alvos: List[IJ] = []
        total = 0
        for lst in capturas_por_direcao:
            total += len(lst)
            alvos.extend(lst)
        self._ultimo_total = total

        if total == 0:
            self._ultimo_modo = "sem_captura"
        elif total <= 2:
            self._ultimo_modo = "direcao_primeiro"
        else:
            self._ultimo_modo = "lado_primeiro"

        # 1. Prepara o payload para a Super-Macro
        payload = {
            "jogada": tuple(jogada),
            "capturas_por_direcao": [
                [tuple(ij) for ij in sublist]
                for sublist in capturas_por_direcao
            ],
            "num_peca_atual": num_peca_atual,
            "speed_ptp": speed_ptp,
            "speed_linear": speed_lin,
            "accel_ptp": accel_ptp,
            "accel_linear": accel_lin
        }

        if self.h: self.h.mensagem(f"--- [R{rid}] Despachando LANCE COMPLETO para o Bridge ---")
        if self.h: self.h.mensagem(f"[R{rid}] Jogada: {tuple(jogada)}, Peça N°: {num_peca_atual}, Flips: {total} (modo: {self._ultimo_modo})")

        # 2. Envia a "Super-Macro" para o bridge
        http_post(
            rid,
            "/macro/executar_lance_completo",
            json=payload,
        )

        # 3. Atualiza o contador de peças no lado do Colab
        self.pecas_usadas[jogador_atual] += 1

        if self.h: self.h.mensagem(f"--- [R{rid}] Bridge confirmou: LANCE COMPLETO concluído ---")

class ErroRobo(Exception):
    pass


# === Ganchos do Juiz (Hooks) ===
def antes_de_aplicar_jogada_robo(jogo, jogada: IJ, jogador_atual: int,
                                 orquestrador: OrquestradorOthelloRobo):
    try:
        if orquestrador.h:
            orquestrador.h.inicio_lance(jogador=jogador_atual, jogada=jogada, jogo=jogo)

        _ = jogo.jogador_atual()
        i, j = jogada
        capturas_por_direcao = jogo._capturas[i][j]

        orquestrador.executar_lance(jogo, jogada, capturas_por_direcao, jogador_atual)

    except Exception as e:
        print(f"!!! ERRO DURANTE O LANCE FÍSICO (Jogador {jogador_atual}) !!!")
        traceback.print_exc()
        raise ErroRobo(f"Falha na execução física (J{jogador_atual}): {e}")


def depois_de_aplicar_jogada_robo(jogo_antes, jogada, jogo_depois, jogador_atual,
                                  orquestrador: OrquestradorOthelloRobo):
    if orquestrador.h:
        orquestrador.h.robo = orquestrador.robos[jogador_atual]
        modo = getattr(orquestrador, "_ultimo_modo", "?")
        total = getattr(orquestrador, "_ultimo_total", 0)
        orquestrador.h.fim_lance(modo=modo, total_flips=total,
                                 jogo_antes=jogo_antes, jogo_depois=jogo_depois)

__all__ = [
    "Historiadora",
    "ControladorRobo",
    "OrquestradorOthelloRobo",
    "ErroRobo",
    "antes_de_aplicar_jogada_robo",
    "depois_de_aplicar_jogada_robo",
    "SPEED_PTP", "SPEED_LINEAR"
]

In [ ]:
import sys, importlib, os
sys.path.insert(0, "/content/drive/MyDrive/TCC_Othello_Juiz")

mod = importlib.import_module("othello_robo_ponte")
importlib.reload(mod)
from othello_robo_ponte import *


Você deve implementar uma inteligência artifical para o jogo.
Sua inteligência artifical deve ser da forma de um objeto que implementa uma interface específica.

Como modelo, considere a classe ```JogadorAleatorio```:

In [ ]:
class JogadorAleatorio():
  def __init__(self):
    pass

  def nova_partida(self, jogo, jogador, id_oponente = None):
    pass

  def escolhe_jogada(self, jogo):
    jogadas_possiveis = jogo.jogadas_legais()
    return jogadas_possiveis[np.random.choice(len(jogadas_possiveis))]

  def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois):
    pass

  def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois):
    pass

  def informa_fim(self, jogo_final):
    pass

  @classmethod
  def cria_jogador(cls):
    return JogadorAleatorio()

Esta é uma classe que faz jogadas puramente aleatórias.

Ela implementa todos os métodos que sua AI deve implementar.

Eles são:


> ```nova_partida(self, jogo, jogador, id_oponente = None)```: Notifica o jogador do início de uma nova partida. O parâmetro ```jogo``` contém o estado inicial e o parâmetro ```jogador``` o número (-1 ou 1) do jogador que este objeto irá representar. O parâmetro opcional ```id_oponente``` é um identificador único do oponente. Cada AI da competição receberá um identificador único.
Você pode usar este identificador para adotar estratégias especializadas contra oponentes específicos.


> ```escolhe_jogada(self, jogo)```: Retorna a próxima jogada do jogador. O parâmetro ```jogo``` é um objeto da classe ```Othello``` com o estado da partida.

> ```informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois)```: Notifica o jogador do resultado de uma jogada feita pelo jogador que ele representa. O parâmetro ```tabuleiro_antes``` contém um objeto da classe ```Othello``` com o estado do jogo *antes* da jogada, o parâmetro ```jogada``` contém a jogada feita e o parâmetro ```tabuleiro_depois``` contém o estado do tabuleiro *depois* da jogada.

> ```informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois)```: Notifica o jogador do resultado de uma jogada feita pelo jogador oponente. O parâmetro ```tabuleiro_antes``` contém um objeto da classe ```Othello``` com o estado do jogo *antes* da jogada, o parâmetro ```jogada``` contém a jogada feita e o parâmetro ```tabuleiro_depois``` contém o estado do tabuleiro *depois* da jogada.

> ```informa_fim(self, jogo_final)```: Notifica o jogador do término do jogo.
O parâmetro ```jogo_final``` contém o estado do jogo ao final da partida.


In [ ]:
import time
from othello_robo_ponte import http_post, ErroRobo

class JogadorHumanoCV():
  def __init__(self, orquestrador, jogador_id):
    self.orq = orquestrador
    self.jogador_id = jogador_id
    self.rid_status = 1

  def nova_partida(self, jogo, jogador, id_oponente=None):
    pass

  def escolhe_jogada(self, jogo):
    print(f"\n--- [HumanoCV] Vez do Humano (Cor: {self.jogador_id}) ---")
    cor_str = "Brancas" if self.jogador_id == -1 else "Pretas"

    try:
        http_post(self.rid_status, "/vis/validar_estado", json=jogo.tabuleiro().tolist())
    except: pass

    http_post(self.rid_status, "/game/set_status", json={
        "jogador": f"Humano ({cor_str})",
        "acao": "Sua vez! Coloque a peça.",
        "jogada": None,
        "passos": []
    })

    # Define o Snapshot Lógico (Zero = Onde pode jogar)
    self.orq.preparar_para_jogada_humana(jogo, self.jogador_id)
    print("[HumanoCV] Aguardando peça...")

    tempo_limite = 120
    inicio = time.time()

    while (time.time() - inicio) < tempo_limite:
        # --- FASE 1: DETECTAR ONDE JOGOU ---
        jogada_tuple = self.orq.checar_jogada_humano(self.jogador_id)

        if jogada_tuple:
            print(f"[HumanoCV] Peça detectada em: {jogada_tuple}")

            if jogo.jogada_legal(jogada_tuple):
                print("[HumanoCV] Jogada Válida. Aguardando humano virar as peças...")

                # 1. Calcula o FUTURO (Como o tabuleiro tem que ficar)
                jogo_futuro = jogo.joga(jogada_tuple)
                tabuleiro_futuro = jogo_futuro.tabuleiro().tolist()

                # 2. Entra num loop até o tabuleiro físico ficar igual ao futuro
                while True:
                    try:
                        resp = http_post(self.rid_status, "/vis/validar_estado", json=tabuleiro_futuro)

                        if resp.get("ok"):
                            http_post(self.rid_status, "/game/set_status", json={
                                "jogador": "Humano", "acao": "Perfeito! Passando a vez...",
                                "jogada": list(jogada_tuple)
                            })
                            time.sleep(1.0)
                            return jogada_tuple

                    except Exception as e:
                        erro_msg = "Complete os movimentos..."
                        try:
                            pass
                        except: pass

                        http_post(self.rid_status, "/game/set_status", json={
                            "jogador": f"Humano ({cor_str})",
                            "acao": "⚠️ VIRE AS PEÇAS (Siga os quadrados vermelhos)",
                            "jogada": list(jogada_tuple)
                        })
                        time.sleep(0.5)

            else:
                # --- CENÁRIO: JOGADA ILEGAL (Mantemos a lógica de limpeza) ---
                print(f"[HumanoCV] ILEGAL: {jogada_tuple}")
                http_post(self.rid_status, "/game/set_status", json={
                    "jogador": "Humano",
                    "acao": f"❌ ILEGAL em {jogada_tuple}! REMOVA A PEÇA.",
                    "jogada": list(jogada_tuple)
                })

                while True:
                    try:
                        http_post(self.rid_status, "/vis/validar_estado", json=jogo.tabuleiro().tolist())
                        break
                    except:
                        time.sleep(0.5)

                # Reset
                http_post(self.rid_status, "/game/set_status", json={
                    "jogador": f"Humano ({cor_str})", "acao": "Tente novamente.", "jogada": None
                })
                self.orq.preparar_para_jogada_humana(jogo, self.jogador_id)

        time.sleep(0.2)

    raise ErroRobo("Timeout: Humano demorou demais.")

  def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois):
    print(f"[HumanoCV] Juiz confirmou minha jogada: {jogada}")
    pass

  def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois):
    # Este é o turno do ROBÔ. Não fazemos nada aqui.
    print(f"[HumanoCV] Oponente (Robô) jogou: {jogada}")
    pass

  def informa_fim(self, jogo_final):
    print(f"[HumanoCV] Fim de jogo reportado pelo Juiz.")
    http_post(self.rid_status, "/game/set_status", json={
        "jogador": "FIM DE JOGO",
        "acao": "Partida Encerrada",
        "jogada": None
    })
    pass

  @classmethod
  def cria_jogador(cls, orquestrador, jogador_id):
    return JogadorHumanoCV(orquestrador, jogador_id)

In [ ]:
gdown.download_folder("https://drive.google.com/drive/folders/16oWy-6vuDp-g8SYNzY_iycGqJ4m_vGhG?usp=sharing", output="jogadores_othello/jogador_exemplo_01")

Agora podemos carregar o módulo:

In [ ]:
def carrega_modulo(modulo, pacote, caminho):
  nome_completo = f"{pacote}.{modulo}"
  # Verifica se modulo ja esta carregado
  if nome_completo not in sys.modules:
    # Verifica se pacote esta carregado
    if pacote not in sys.modules:
      # Verifica se existe __init__.py
      caminho_pacote = f"{caminho}/{pacote}/__init__.py"
      spec_pacote =  importlib.util.spec_from_file_location(pacote, caminho_pacote)
      if spec_pacote:
        modulo_pacote = importlib.util.module_from_spec(spec_pacote)

        spec_pacote.loader.exec_module(modulo_pacote)
        sys.modules[pacote] = modulo_pacote
    # Carrega modulo
    caminho_modulo = f"{caminho}/{pacote}/{modulo}.py"
    spec_modulo =  importlib.util.spec_from_file_location(nome_completo, caminho_modulo)
    modulo = importlib.util.module_from_spec(spec_modulo)
    spec_modulo.loader.exec_module(modulo)
    sys.modules[nome_completo] = modulo
  return sys.modules[nome_completo]

In [ ]:
ai_exemplo_01 = carrega_modulo("jogador", "jogador_exemplo_01", "/content/jogadores_othello")

Observe que este módulo expõe a função ```cria_jogador```:

In [ ]:
ai_exemplo_01.cria_jogador

Vamos criar um jogador:

In [ ]:
jogador_exemplo_01 = ai_exemplo_01.cria_jogador()

Observe que este método exporta os métodos necessários:

In [ ]:
for nome in dir(jogador_exemplo_01):
  if not nome.startswith("_"):
    print(nome)

Podemos testar outros jogadores.


In [ ]:
# Você pode trocar essa URL por uma URL de um jogador seu para testá-lo
%cd /content/

teste_url = "https://drive.google.com/drive/folders/1c1NLt-b6oiAFXU3Xlvt1r5NUbwCVba68?usp=sharing"


gdown.download_folder(teste_url, output="jogadores_othello/jogador_teste_01")
ai_teste_01 = carrega_modulo("jogador", "jogador_teste_01", "/content/jogadores_othello")

In [ ]:
%cd /content/

# Você pode trocar essa URL por uma URL de um jogador seu para testá-lo
teste_url2 = "https://drive.google.com/drive/folders/1cV6ZMcpFnv8_PV7-dtJhVLdOwr_YVfdu?usp=sharing"

gdown.download_folder(teste_url, output="jogadores_othello/jogador_teste_02")
ai_teste_02 = carrega_modulo("jogador", "jogador_teste_02", "/content/jogadores_othello")

### Regras para sua AI

A sua AI será executada no ambiente Google Colab com acelerador GPU.
Você *não* deve criar processos adicionais na máquina virtual, nem comunicar-se com outros computadores remotos.

O seu módulo deve ter no máximo 100Mb de tamanho.

Você tem *limites de tempo* para executar cada tarefa.
Os limites são:



*   Criação de jogador (via ```criar_jogador()```): 1000 millisegundos
*   Notificação de nova partida (via ```nova_partida```): 150 millisegundos
*   Escolha de uma jogada (via ```escolhe_jogada```): 150 millisegundos
*   Notificação de jogada própria (via ```informe_propria_jogada```): 150 millisegundos
*   Notificação de jogada do oponente (via ```informe_jogada_oponente```): 150 millisegundos
*   Notificação de término de partida (via ```informa_fim```): 3000 millisegundos

Estes limites serão *rígidos*.
Se durante uma partida seu jogador violar algum deles, será considerado derrotado.

A classe ```JogadorProxy``` envelopa um jogador e controla esses tempos executando cada método em uma *thread* com timeout:


In [ ]:
class ErroJogador(Exception):
  pass

class ErroJogadorTimeout(ErroJogador):
  pass

class JogadorProxy():
  timeout_criacao = 1.0
  timeout_nova_partida = 0.15
  timeout_jogada = 0.15
  timeout_notificacao_jogada = 0.15
  timeout_notificacao_fim = 3
  timeout_kill = 0.1

  @classmethod
  def _thread_func_wrapper(cls, func, ret, args):
    try:
      res = func(*args)
    except Exception as ex:
      ret[1] = ex
    else:
      ret[0] = res

  def _exec_thread(self, func, timeout, *args):
    result = [None, None]
    thread = threading.Thread(target=JogadorProxy._thread_func_wrapper, args=(func, result, args))
    thread.start()
    thread.join(timeout=timeout)
    if thread.is_alive():
      # Timeout
      # Tenta matar a thread
      ctypes.pythonapi.PyThreadState_SetAsyncExc(thread.native_id, ctypes.py_object(SystemExit))
      thread.join(timeout=JogadorProxy.timeout_kill)
      raise ErroJogadorTimeout()
    if result[1]: # Ocorreu exceção
      raise ErroJogador from result[1]
    return result[0]

  def __init__(self, modulo):
    self._jogador = self._exec_thread(modulo.cria_jogador, JogadorProxy.timeout_criacao)

  def nova_partida(self, jogo, jogador, id_oponente = None):
    return self._exec_thread(self._jogador.nova_partida, JogadorProxy.timeout_nova_partida, jogo, jogador, id_oponente)

  def escolhe_jogada(self, jogo):
    return self._exec_thread(self._jogador.escolhe_jogada, JogadorProxy.timeout_jogada, jogo)

  def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois):
    return self._exec_thread(self._jogador.informa_propria_jogada, JogadorProxy.timeout_notificacao_jogada, tabuleiro_antes, jogada, tabuleiro_depois)

  def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois):
    return self._exec_thread(self._jogador.informa_jogada_oponente, JogadorProxy.timeout_notificacao_jogada, tabuleiro_antes, jogada, tabuleiro_depois)

  def informa_fim(self, jogo_final):
    return self._exec_thread(self._jogador.informa_fim, JogadorProxy.timeout_notificacao_fim, jogo_final)


A função ```compara_ais``` realiza partidas nos moldes das da competição.
Ela retorna 1 se a primeira ai for considerada vencedora, -1 se a segunda AI for considerada vencedora e 0 se houver empate.

In [ ]:
%cd /content/drive/MyDrive/TCC_Othello_Juiz
import sys; sys.path.insert(0, '.')

from othello_robo_ponte import ErroRobo

def compara_ais(primeiro_modulo, segundo_modulo, partidas, progresso_func = None, erro_func = None, antes_de_aplicar = None, depois_de_aplicar = None):
  sequencia_partidas = np.random.default_rng().permutation(partidas)
  try:
    jogador1 = JogadorProxy(primeiro_modulo)
  except ErroJogador:
    if erro_func:
      erro_func("Impossível criar jogador", 0)
      return 1
  try:
    jogador2 = JogadorProxy(segundo_modulo)
  except ErroJogador:
    if erro_func:
      erro_func("Impossível criar jogador", 1)
      return -1
  jogadores_col = [{1:jogador1, -1:jogador2}, {1:jogador2, -1:jogador1}]
  placar = [0, 0]
  for j, i in enumerate(sequencia_partidas):
    pula_avaliacao = False
    jogo = Othello()
    ii = i%2
    jogadores = jogadores_col[ii]
    try:
      jogadores[1].nova_partida(jogo, 1)
    except ErroJogador:
      if erro_func:
        erro_func("Falha ao notificar nova partida", ii)
      placar[1-ii] += 1
      continue
    try:
      jogadores[-1].nova_partida(jogo, -1)
    except ErroJogador:
      if erro_func:
        erro_func("Falha ao notificar nova partida", 1-ii)
      placar[ii] += 1
      continue
    while not jogo.terminou():
      i_jogador_atual = jogo.jogador_atual()
      i_placar = (((i_jogador_atual+3)//2)+ii)%2
      jogador_atual = jogadores[i_jogador_atual]
      oponente = jogadores[-i_jogador_atual]
      try:
        jogada = jogador_atual.escolhe_jogada(jogo)

        if antes_de_aplicar is not None:
          _ = jogo.jogador_atual()              # garante capturas atualizadas
          antes_de_aplicar(jogo, jogada, i_jogador_atual)

        jogo_depois = jogo.joga(jogada)
        jogador_atual.informa_propria_jogada(jogo, jogada, jogo_depois)
      except (ErroJogador, ErroRobo):
        if erro_func:
            erro_func("Falha ao fazer jogada/notificar resultado", i_placar)
        placar[1 - i_placar] += 1
        pula_avaliacao = True
        break
      try:
        oponente.informa_jogada_oponente(jogo, jogada, jogo_depois)
      except ErroJogador:
        if erro_func:
          erro_func("Falha ao fazer jogada/notificar resultado", 1-i_placar)
        placar[i_placar] += 1
        pula_avaliacao = True
        break
      if depois_de_aplicar is not None:
        depois_de_aplicar(jogo_antes=jogo, jogada=jogada, jogo_depois=jogo_depois, jogador_atual=i_jogador_atual)
      jogo = jogo_depois
      if progresso_func:
        progresso_func(j+1, placar)
    if pula_avaliacao:
      continue
    try:
      jogadores[1].informa_fim(jogo)
    except ErroJogador:
      if erro_func:
        erro_func("Falha ao notificar fim", ii)
      placar[1-ii] += 1
      continue
    try:
      jogadores[-1].informa_fim(jogo)
    except ErroJogador:
      if erro_func:
        erro_func("Falha ao notificar fim", 1-ii)
      placar[ii] += 1
      continue

    if jogo.placar(1)>32:
      placar[ii] += 1
    elif jogo.placar(1)<32:
      placar[1-ii] += 1

  if placar[0]>placar[1]:
    return 1
  elif placar[1]>placar[0]:
    return -1
  return 0

In [ ]:
import traceback

def executar_partida_humano_vs_robo(
    jogador_preto,
    jogador_branco,
    jogador_preto_id,
    jogador_branco_id,
    antes_de_aplicar,
    depois_de_aplicar,
    erro_func
):
    """
    Uma versão simplificada do 'compara_ais' que NÃO usa JogadorProxy
    e aceita instâncias de jogador já criadas, permitindo
    que o humano tenha tempo ilimitado para jogar.
    """

    jogo = Othello()

    # Mapeia ID -> instância
    jogadores = {
        jogador_preto_id: jogador_preto,
        jogador_branco_id: jogador_branco
    }

    # Notifica o início da partida para ambos (sem proxy)
    try:
        print("[Partida] Notificando início para o Jogador Preto...")
        jogador_preto.nova_partida(jogo, jogador_preto_id, id_oponente='humano')
        print("[Partida] Notificando início para o Jogador Branco (Humano)...")
        jogador_branco.nova_partida(jogo, jogador_branco_id, id_oponente='robo')
    except Exception as e:
        erro_func("Falha ao notificar nova partida", e)
        traceback.print_exc()
        return 0

    # Loop principal do jogo
    while not jogo.terminou():
        i_jogador_atual = jogo.jogador_atual()
        jogador_atual_obj = jogadores[i_jogador_atual]
        oponente_obj = jogadores[-i_jogador_atual]

        try:
            jogada = jogador_atual_obj.escolhe_jogada(jogo)

            if antes_de_aplicar:
                _ = jogo.jogador_atual()
                antes_de_aplicar(jogo, jogada, i_jogador_atual)

            jogo_depois = jogo.joga(jogada)

            # Notifica ambos (sem proxy)
            jogador_atual_obj.informa_propria_jogada(jogo, jogada, jogo_depois)
            oponente_obj.informa_jogada_oponente(jogo, jogada, jogo_depois)

            if depois_de_aplicar:
                depois_de_aplicar(jogo_antes=jogo, jogada=jogada, jogo_depois=jogo_depois, jogador_atual=i_jogador_atual)

            # Atualiza o estado do jogo para o próximo loop
            jogo = jogo_depois

        except (ErroRobo, Exception) as e:
            # Se o humano der timeout (o nosso de 2min) ou o robô falhar
            print(f"!!! ERRO FATAL DURANTE O LANCE (Jogador {i_jogador_atual}) !!!")
            traceback.print_exc()
            erro_func(f"Falha na jogada do J{i_jogador_atual}", e)
            return -i_jogador_atual # O outro jogador vence

    # --- Fim de Jogo ---
    print("[Partida] Jogo terminou. Notificando jogadores...")
    try:
        jogador_preto.informa_fim(jogo)
        jogador_branco.informa_fim(jogo)
    except Exception as e:
        erro_func("Falha ao notificar fim de jogo", e)

    # Determina o vencedor
    placar_preto = jogo.placar(jogador_preto_id)
    placar_branco = jogo.placar(jogador_branco_id)

    print(f"\n--- FIM DA PARTIDA ---")
    print(f"Placar: Pretas ({placar_preto}) vs Brancas ({placar_branco})")

    if placar_preto > placar_branco:
        return jogador_preto_id # Preto venceu
    elif placar_branco > placar_preto:
        return jogador_branco_id # Branco venceu
    else:
        return 0 # Empate

In [ ]:
def relata_erro(msg, jogador):
  print("Erro: " + msg)
  print("Jogador responsável: " + str(jogador))

def relata_progresso(i, placar):
  print(f"\r{100*i/200}%: {placar}", end="")

In [ ]:
import traceback

from othello_robo_ponte import (
    ControladorRobo, OrquestradorOthelloRobo,
    antes_de_aplicar_jogada_robo, depois_de_aplicar_jogada_robo, Historiadora,
    SPEED_PTP, SPEED_LINEAR
)

hist = Historiadora(imprimir=True, verbosidade=1, mostrar_matriz=True)

# Define os RIDs para os jogadores
ROBO_JOGADOR_1 = 1
ROBO_JOGADOR_2 = 2

robo1 = None
robo2 = None

try:
    # --- INICIALIZA ROBÔ 1 (PRETAS / Jogador 1) ---
    robo1 = ControladorRobo(
        rid=ROBO_JOGADOR_1,
        speed_ptp=SPEED_PTP,
        speed_linear=SPEED_LINEAR,
        historico=hist
    )
    robo1.conectar()
    robo1.habilitar_servos()
    print("--- Robô 1 (Pretas) pronto. ---")

    # --- INICIALIZA ROBÔ 2 (BRANCAS / Jogador -1) ---
    robo2 = ControladorRobo(
        rid=ROBO_JOGADOR_2,
        speed_ptp=SPEED_PTP,
        speed_linear=SPEED_LINEAR,
        historico=hist
    )
    robo2.conectar()
    robo2.habilitar_servos()
    print("--- Robô 2 (Brancas) pronto. ---")

    # Inicializa o orquestrador com AMBOS os robôs
    orq = OrquestradorOthelloRobo(robo1=robo1, robo2=robo2, historico=hist, usar_camera=False)

    # Manda ambos para o Home antes de começar
    print("Enviando R1 para o Home...")
    orq.robos[1].ir_home()
    print("Enviando R2 para o Home...")
    orq.robos[-1].ir_home()
    print("--- Ambos os robôs estão no Home. Iniciando partida. ---")


    # Ganchos SIMPLES.
    def gancho_antes(jogo, jogada, jogador_atual):
        return antes_de_aplicar_jogada_robo(jogo, jogada, jogador_atual, orq)

    def gancho_depois(jogo_antes, jogada, jogo_depois, jogador_atual):
        # Este gancho apenas loga o resultado.
        return depois_de_aplicar_jogada_robo(jogo_antes, jogada, jogo_depois, jogador_atual, orq)

    # Roda a partida
    resultado = compara_ais(
        ai_teste_02, ai_teste_01, 1,
        progresso_func=relata_progresso,
        erro_func=relata_erro,
        antes_de_aplicar=gancho_antes,
        depois_de_aplicar=gancho_depois
    )

except Exception as e:
    print(f"ERRO FATAL NO JUIZ: {e}")
    traceback.print_exc()

finally:
    print("\nDesligando servos de AMBOS os robôs...")
    try:
        if robo1:
            robo1.desabilitar_servos()
            print(f"Servos Robô {ROBO_JOGADOR_1} desligados.")
    except Exception as e:
        print(f"Falha ao desligar R{ROBO_JOGADOR_1}: {e}")
    try:
        if robo2:
            robo2.desabilitar_servos()
            print(f"Servos Robô {ROBO_JOGADOR_2} desligados.")
    except Exception as e:
        print(f"Falha ao desligar R{ROBO_JOGADOR_2}: {e}")

In [ ]:
import traceback

from othello_robo_ponte import (
    ControladorRobo, OrquestradorOthelloRobo,
    antes_de_aplicar_jogada_robo, depois_de_aplicar_jogada_robo, Historiadora,
    SPEED_PTP, SPEED_LINEAR, ACCEL_PTP, ACCEL_LINEAR
)

hist = Historiadora(imprimir=True, verbosidade=1, mostrar_matriz=True)

ROBO_JOGADOR_1 = 1
ROBO_JOGADOR_2 = 2

robo1 = None
robo2 = None

try:
    # --- INICIALIZA ROBÔS ---
    robo1 = ControladorRobo(
        rid=ROBO_JOGADOR_1,
        speed_ptp=SPEED_PTP, speed_linear=SPEED_LINEAR,
        accel_ptp=ACCEL_PTP, accel_linear=ACCEL_LINEAR,
        historico=hist
    )
    robo1.conectar()
    robo1.habilitar_servos()
    print("--- Robô 1 (Pretas) pronto. ---")

    robo2 = ControladorRobo(
        rid=ROBO_JOGADOR_2,
        speed_ptp=SPEED_PTP, speed_linear=SPEED_LINEAR,
        accel_ptp=ACCEL_PTP, accel_linear=ACCEL_LINEAR,
        historico=hist
    )
    robo2.conectar()
    robo2.habilitar_servos()
    print("--- Robô 2 (Brancas) pronto. ---")

    orq = OrquestradorOthelloRobo(robo1=robo1, robo2=robo2, historico=hist)

    print("Enviando R1 para o Home...")
    orq.robos[1].ir_home()
    print("Enviando R2 para o Home...")
    orq.robos[-1].ir_home()
    print("--- Ambos os robôs estão no Home. Iniciando partida. ---")

    # --- DEFINIÇÃO DOS JOGADORES ---
    JOGADOR_ROBO_ID = 1
    JOGADOR_HUMANO_ID = -1

    # 1. Cria a instância do jogador Robô
    print("[Partida] Criando instância do Jogador Robô (IA)...")
    jogador_robo_inst = JogadorProxy(JogadorAleatorio)

    # 2. Cria a instância do jogador Humano
    print("[Partida] Criando instância do Jogador Humano (CV)...")
    jogador_humano_inst = JogadorHumanoCV.cria_jogador(orquestrador=orq, jogador_id=JOGADOR_HUMANO_ID)


    # --- GANCHOS (Hooks) ---
    def gancho_antes(jogo, jogada, jogador_atual):
        if jogador_atual == JOGADOR_ROBO_ID:
            # É a vez do robô, executa o movimento
            return antes_de_aplicar_jogada_robo(jogo, jogada, jogador_atual, orq)
        else:
            # É a vez do humano, o movimento já foi feito (na câmera)
            if orq.h:
                orq.h.inicio_lance(jogador=jogador_atual, jogada=jogada, jogo=jogo)

    def gancho_depois(jogo_antes, jogada, jogo_depois, jogador_atual):
        return depois_de_aplicar_jogada_robo(jogo_antes, jogada, jogo_depois, jogador_atual, orq)


    print(f"--- Iniciando Partida: Robô (Pretas, J1) vs Humano (Brancas, J-1) ---")

    # 3. Chama a NOVA função de partida
    resultado_final = executar_partida_humano_vs_robo(
        jogador_preto=jogador_robo_inst,
        jogador_branco=jogador_humano_inst,
        jogador_preto_id=JOGADOR_ROBO_ID,
        jogador_branco_id=JOGADOR_HUMANO_ID,
        antes_de_aplicar=gancho_antes,
        depois_de_aplicar=gancho_depois,
        erro_func=relata_erro
    )

    if resultado_final == JOGADOR_ROBO_ID: print("Vitória do Robô!")
    elif resultado_final == JOGADOR_HUMANO_ID: print("Vitória do Humano!")
    else: print("Empate!")

except Exception as e:
    print(f"ERRO FATAL NO JUIZ: {e}")
    traceback.print_exc()

finally:
    print("\nDesligando servos de AMBOS os robôs...")
    try:
        if robo1:
            robo1.desabilitar_servos()
            print(f"Servos Robô {ROBO_JOGADOR_1} desligados.")
    except Exception as e:
        print(f"Falha ao desligar R{ROBO_JOGADOR_1}: {e}")
    try:
        if robo2:
            robo2.desabilitar_servos()
            print(f"Servos Robô {ROBO_JOGADOR_2} desligados.")
    except Exception as e:
        print(f"Falha ao desligar R{ROBO_JOGADOR_2}: {e}")